In [ ]:
import os 
drive_root = os.getcwd() + '/drive/MyDrive/eecs545project-main'
%cd $drive_root

/content/drive/MyDrive/eecs545project-main


In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader
from torchsummary import summary
import torchvision.models as models
from PIL import Image
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.10.0+cu111
Torchvision Version:  0.11.1+cu111


In [ ]:
# First define some parameters
model_name = "resnet_fcn"
num_classes = 20
batch_size = 8
num_epochs = 50
feature_extract = True

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in zip(dataloaders[phase][0], dataloaders[phase][1]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)["out"]
                # print(outputs.shape)
                # print(labels.squeeze(0).shape)
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # backward + optimize only if in training phase
                    if phase == 'train':
                      loss = criterion(outputs, labels)
                      loss.backward()
                      optimizer.step()

                # statistics
                preds = model(inputs)["out"].argmax(1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase][0].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase][0].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet_fcn":
        """ Resnet fcn model from pytorch
        """
        model_ft = models.segmentation.fcn_resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        #first, change auxiliary
        model_ft.aux_classifier = models.segmentation.fcn.FCNHead(1024, 13)
        #next, the classifier
        model_ft.classifier = models.segmentation.fcn.FCNHead(2048, 13)
        input_size = 256
    
    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
#print(model_ft)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

In [ ]:
params_to_update = model_ft.parameters()
print("Params to learn:")
if True:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 classifier.0.weight
	 classifier.1.weight
	 classifier.1.bias
	 classifier.4.weight
	 classifier.4.bias
	 aux_classifier.0.weight
	 aux_classifier.1.weight
	 aux_classifier.1.bias
	 aux_classifier.4.weight
	 aux_classifier.4.bias


In [ ]:
 test = Image.open(os.path.join(os.getcwd(),'segment/cmp_b0001.png'))

In [ ]:
transform_list = [transforms.ToTensor()]
transform = transforms.Compose(transform_list)
torch.round(transform(test)*255).long().dtype

torch.int64

In [ ]:
# Next, define a dictionary of dataloaders for the training and validation
from facades_dataset import FacadeDataset
input_truth = FacadeDataset(os.getcwd() + '/input_images', False)
segment = FacadeDataset(os.getcwd() + '/segment', True)
train_dataset_truth, val_dataset_truth, test_dataset_truth = torch.utils.data.random_split(input_truth, [250, 64, 64], generator=torch.Generator().manual_seed(42))
train_dataset_segment, val_dataset_segment, test_dataset_segment = torch.utils.data.random_split(segment, [250, 64, 64], generator=torch.Generator().manual_seed(42))


train_loader_truth = DataLoader(train_dataset_truth, batch_size = 1)
train_loader_segment = DataLoader(train_dataset_segment, batch_size=1)
val_loader_truth = DataLoader(val_dataset_truth, batch_size = 1)
val_loader_segment = DataLoader(val_dataset_segment, batch_size = 1)
dataloaders_dict = {'train':[train_loader_truth, train_loader_segment], 'val':[val_loader_truth, val_loader_segment]}

In [ ]:
torch.max(next(iter(train_dataset_segment)))

tensor(11)

In [ ]:
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/49
----------
train Loss: 1.8286 Acc: 26735.6400
val Loss: 1.8348 Acc: 25434.8750

Epoch 1/49
----------
train Loss: 1.7280 Acc: 27868.3840
val Loss: 1.7857 Acc: 24929.0000

Epoch 2/49
----------
train Loss: 1.6995 Acc: 28187.5720
val Loss: 1.7631 Acc: 25056.9375

Epoch 3/49
----------
train Loss: 1.6766 Acc: 28493.7680
val Loss: 1.7396 Acc: 25055.7500

Epoch 4/49
----------
train Loss: 1.6558 Acc: 28822.8520
val Loss: 1.7162 Acc: 25196.7500

Epoch 5/49
----------
train Loss: 1.6354 Acc: 29226.0920
val Loss: 1.6948 Acc: 25205.6406

Epoch 6/49
----------
train Loss: 1.6140 Acc: 29679.1640
val Loss: 1.6699 Acc: 25168.9219

Epoch 7/49
----------
train Loss: 1.5922 Acc: 30200.3840
val Loss: 1.6491 Acc: 24946.0781

Epoch 8/49
----------
train Loss: 1.5690 Acc: 30778.1080
val Loss: 1.6187 Acc: 24724.6250

Epoch 9/49
----------
train Loss: 1.5446 Acc: 31427.3800
val Loss: 1.6064 Acc: 24240.8125

Epoch 10/49
----------
train Loss: 1.5201 Acc: 32136.3200
val Loss: 1.5828 Acc: 23777.9844

In [ ]:
torch.save(model_ft.state_dict(), os.path.join(os.getcwd(), 'fcn_save_50epoch'))

NameError: ignored

In [ ]:
model_ft.eval()
model_ft.cuda()
output_sample = model_ft(next(iter(train_dataset_truth)).unsqueeze(0).to(device))["out"]
plt.imshow(output_sample.squeeze(0).argmax(0).cpu())
print(output_sample.squeeze(0).argmax(0))